# X2Paddle快速上手——PyTorch迁移至PaddlePaddle
***X2Paddle简介***：X2Paddle支持将Caffe/TensorFlow/ONNX/PyTorch深度学习框架训练得到的模型，迁移至PaddlePaddle模型。   
***X2Paddle代码GitHub链接***：[https://github.com/PaddlePaddle/X2Paddle](https://github.com/PaddlePaddle/X2Paddle)  
***【注意】***前往GitHub给[X2Paddle](https://github.com/PaddlePaddle/X2Paddle)点击Star，关注项目，即可随时了解X2Paddle的最新进展。  
本教程用于帮助用户学习将PyTorch训练后的预测模型迁移至PaddlePaddle框架，以PyTorch版本的[AlexNet](https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py)为例进行详细介绍。  

## 安装及准备
### 1. 安装X2Paddle
***方式一：（推荐）***

In [ ]:
! git clone https://github.com/PaddlePaddle/X2Paddle.git
! cd X2Paddle
! git checkout develop
! python setup.py install

***方式二：***

In [ ]:
! pip install x2paddle==1.0.1 --index https://pypi.Python.org/simple/

### 2. 安装PyTorch

In [ ]:
! pip install torch==1.6.0 torchvision==0.7.0

### 3. 安装PaddlePaddle

In [ ]:
! pip install paddlepaddle==2.0.1

## 模型迁移
### 1. 获取AlexNet模型

In [ ]:
from torchvision.models import AlexNet
from torchvision.models.utils import load_state_dict_from_url

torch_model = AlexNet()
torch_state_dict = load_state_dict_from_url('https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth')
torch_model.load_state_dict(torch_state_dict)
torch_model.eval()

### 2. 转换
PyTorch到PaddlePaddle的转换需要传入输入的示例，才可以进行转换，以下为构建输入的过程（输入也可为值随机初始化的Tensor）：

In [ ]:
import numpy as np
from PIL import Image
img = Image.open("dog_pt.png")
img = np.array(img).astype("float32") / 255.0
img -= [0.485, 0.456, 0.406]
img /= [0.229, 0.224, 0.225]
img = np.transpose(img, (2, 0, 1))
img = np.expand_dims(img, 0)

import torch
input_tensor = torch.tensor(img)

***方式一：*** Trace方式  
模型输入的shape固定，PyTorch模型基本均支持此方式转换。

In [ ]:
from x2paddle.convert import pytorch2paddle
save_dir = "pd_model_trace"
jit_type = "trace"
pytorch2paddle(module=torch_model, 
               save_dir=save_dir, 
               jit_type=jit_type, 
               input_examples=[input_tensor])
# module (torch.nn.Module): PyTorch的Module。
# save_dir (str): 转换后模型的保存路径。
# jit_type (str): 转换方式，此时为"trace"。
# input_examples (list[torch.tensor]): torch.nn.Module的输入示例，list的长度必须与输入的长度一致。默认为None。

***方式二：*** Script方式  
模型输入的shape可不固定，由于PyTorch的Script方式可识别的代码格式有限，所以PyTorch模型在此方式下转换的支持度较低。

In [ ]:
from x2paddle.convert import pytorch2paddle
save_dir = "pd_model_script"
jit_type = "script"
pytorch2paddle(module=torch_model, 
               save_dir=save_dir, 
               jit_type=jit_type, 
               input_examples=[input_tensor])
# module (torch.nn.Module): PyTorch的Module。
# save_dir (str): 转换后模型的保存路径。
# jit_type (str): 转换方式，此时为"script"。
# input_examples (list[torch.tensor]): torch.nn.Module的输入示例，list的长度必须与输入的长度一致。默认为None。

使用方式一转换的PaddlePaddle预测模型进行预测：  
（1）下载ImageNet类别文件

In [ ]:
! wget https://raw.githubusercontent.com/Lasagne/Recipes/master/examples/resnet50/imagenet_classes.txt

（2）预测

In [ ]:
import paddle
paddle.enable_static()
exe = paddle.static.Executor(paddle.CPUPlace())
[prog, inputs, outputs] = paddle.static.load_inference_model(path_prefix="pd_model_trace/inference_model", 
                                                            executor=exe, 
                                                            model_filename="model.pdmodel",
                                                            params_filename="model.pdiparams")
print(img.shape)
result = exe.run(prog, feed={inputs[0]: img}, fetch_list=outputs)
max_index = np.argmax(result)
with open('imagenet_classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]
print("The category of dog.jpg is: {}".format(classes[max_index]))